In [1]:
import yt_dlp
from pytubefix import Search, YouTube
import whisper
import torch
import os
import re

In [2]:
# Initialize Whisper model and check for CUDA support
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model = whisper.load_model("turbo", device=device)

Using device: cuda


/home/youssef/.miniconda3/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device

In [3]:
def format_time_to_srt(seconds):
    """Helper function to convert time in seconds to SRT format (HH:MM:SS,MS)"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{int(seconds):02},{milliseconds:03}"

In [ ]:
def transcribe_with_whisper(video_title, model, language="en"):
    # Transcription function with SRT output
    result = model.transcribe(
        f"audio/{video_title}.wav",
        language=language,  # Language specified in global variable
        task="transcribe",
        fp16=torch.cuda.is_available(),  # Use float16 if on GPU
        verbose=False
    )

    # Extract the segments with timestamps and text
    segments = result["segments"]

    # Prepare the SRT formatted transcription
    srt_transcription = ""
    for i, segment in enumerate(segments, 1):
        start_time = segment["start"]
        end_time = segment["end"]
        text = segment["text"]
        
        # Convert times from seconds to SRT format (HH:MM:SS,MS)
        start_time_srt = format_time_to_srt(start_time)
        end_time_srt = format_time_to_srt(end_time)
        
        # Append each segment in SRT format (ensuring correct spacing)
        # print(text)
        srt_transcription += f"{i}\n{start_time_srt} --> {end_time_srt}\n{text}\n\n"

    # Remove the trailing newline after the last subtitle (to avoid extra blank line)
    srt_transcription = srt_transcription.strip()

    # Save the SRT transcription to a file
    transcription_path = f"whisper_captions/{video_title}.srt"
    with open(transcription_path, "w",encoding="utf-8") as f:
        f.write(srt_transcription)  # Write without extra newlines
    
    print(f"SRT transcription generated with Whisper for {video_title}.")
    return transcription_path

In [5]:
def download_audio_and_transcription(query, num_videos=1, language="en"):
    #check if the audio file exists in the directory
    if not os.path.exists("audio"):
        os.makedirs("audio")
    if not os.path.exists("YT_captions"):
        os.makedirs("YT_captions")
    if not os.path.exists("whisper_captions"):
        os.makedirs("whisper_captions")

    # Search for the video
    search = Search(query)
    videos = search.videos  # Use .videos instead of .results
    if not videos:
        print("No videos found for query.")
        return
    
    processed_count = 0
    i = 0  # Index for the video list
    
    # Iterate until we process the required number of videos
    while processed_count < num_videos:
        if i >= len(videos):  # If there are not enough videos, restart search or end
            print("Not enough videos found, stopping after processing.")
            break
        
        
        video = videos[i]
        yt = YouTube(video.watch_url)
        
        video_url = video.watch_url
        # Remove all invalid characters in a file name using regular expressions
        video_title = re.sub(r"[^\w]", "_", video.title)

        # Download the audio
        audio_path = f"audio/{video_title}.wav"
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': f"audio/{video_title}.%(ext)s",  # Adjust the template to avoid double extensions
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            }],
            'quiet': True,
        }

        # Get the video length in minutes
        video_length = yt.length / 60  # Convert seconds to minutes
        
        # Check if the video length is within the desired range (5 to 15 minutes)
        if video_length < 5 or video_length > 15:
            print(f"Skipping video '{yt.title}' (Length: {int(video_length)} minutes)")
            i += 1  # Move to the next video
            continue
        if os.path.exists(audio_path):
            print(f"Audio for '{video_title}' already exists. Skipping download.")
            i += 1 # Move to the next video
            continue
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print(f"Downloading audio for {video_title}...")
            ydl.download([video_url])
        
        # Check for YouTube transcription
        captions = video.captions
        if captions:
            for lang in captions:
                if lang.code == language:
                    transcript = lang.generate_srt_captions()
                    transcription_path = f"YT_captions/{video_title}.srt"
                    with open(transcription_path, "w",encoding="utf-8") as f:
                        f.write(transcript)
                    print(f"Downloaded YouTube transcription for {video_title}.")
                    break
            else:
                print(f"No language transcription available for {video_title}, generating with Whisper...")
                transcription_path = transcribe_with_whisper(video_title, model, language)
        else:
            print(f"No transcription available for {video_title}, generating with Whisper...")
            transcription_path = transcribe_with_whisper(video_title, model, language)
        
        print(f"Audio and transcription saved for {video_title}.")
        processed_count += 1
        i += 1

    print(f"Processed {num_videos} videos.")

In [ ]:
# Usage
query = "دروس اونلاين"
num_videos = 2  # Modify this number to download and transcribe N videos
language = "ar"  # Language code for YouTube transcription (if available)
download_audio_and_transcription(query, num_videos, language)

Skipping video 'هذا الكتاب خسرني مليون جنيه' (Length: 22 minutes)
Skipping video 'ذاكر معي ٣ ساعات بتقنية البومودورو  - Study with me' (Length: 180 minutes)
Skipping video 'THE BEST MM2 YOUTUBER DUOS 🔥' (Length: 0 minutes)
No transcription available for مستحيل_تنسى_الفرق_بين_Do_Does__Did___قواعد_اللغة_الإنجليزية, generating with Whisper...


100%|██████████| 35484/35484 [00:14<00:00, 2457.33frames/s]


 مرحبا بكم أيها الرائعون في قناتكم
 English Seat لتعلم اللغة الإنجليزية
 في درس اليوم سوف أشرح الفرق بين
 Do, Does and Did
 هناك الكثير من المتعلمين للغة الإنجليزية
 يتساءلون عن الفرق بين هذه الكلمات
 وخاصة المبتدئين في تعلم اللغة الإنجليزية
 إذن لكي تفهموا هذا الدرس بشكل واضح
 شاهدوه حتى النهاية لكي تستفيدوا منه بشكل جيد
 إذن شرط أن تشاهدوا هذا الدرس حتى النهاية
 وأعيدكم أنكم سوف تفهمون بشكل واضح
 ولن يبقى لكم أي أسئلة
 إذن بالنسبة لي Do, Does and Did
 هذه الكلمات جاءت من مصدر
 To do وهو أن يفعل أو أن يصنع
 To do
 إذن Do, Does and Did جاءت من To do وهو مصدر
 إذن متى نستعمل Do و Does و Did
 نستعمل Do و Does في الحاضر البسيط
 في الزمن الحاضر
 إذن بالنسبة للزمن الحاضر
 فإننا نقول I do
 أنا أفعل
 إذن دائما نتحدد هنا في Present Simple في الحاضر
 إذن حينما أريد أن أصرف To do في الحاضر
 أقول I do, You do, He does, She does, It does, We do, You do and They do
 إذن هذا فيما يتعلق بالزمن الحاضر
 نصرف To do إلى I do, You do, He does, She does, It does, We do, You do and They do
 لاحظوا معي جيدا 

100%|██████████| 69706/69706 [01:50<00:00, 631.91frames/s] 

ärt رائ 2021 وحمد و Mi أليس سطفان
 وحنا بزيار
 يغ Nilسي قرر
 إنه ع 있던枠
 لأنه يصبح مجدس
 ماذا؟
 ماذا؟
 ماذا؟
 ماذا؟
 ماذا؟
 ماذا؟
 في التعامل المعبادة، يكون مباشرة على الهدف
 ماذا؟
 ماذا؟
 ماذا؟
 تحديده، تحديده، وماذا؟
 تحود لك
 متى محافظه τη حتى
 لا المرجال تحديده المجاعدم
 سظر لك
 واجهت الى فرور
 وفورد وفورد
 جيدا
 هناك فتاة
 وفورد
 وفورد
 وفورد المساعدة
 وفورد المساعدة
 فتاة
 وفورد
 وفورد
 وفورد
 وفورد
 وفورد
 وفورد
 فتاة
 وفورد
 بحظ stone
 وفورد
 ايز الجح Looking,離聾
 لستخدم
 مال對吧
 أماكن
 واحدة
 أنا
 وهوamızzić llega
 سألم انهaign سيمتها
 ي arcade.
 لان قد مرد إليك sided dating.
 سنعط communismا للجوم!
 ع Sa Late.
 فضة الأخسائق اللع other than that.
 انسع PH Federation.
 عنا!
��Son.
فيديلما.
 حسناً ص GT 434 tapi مرة 690.
ًحسناً للعبق Jessica.
؟
 فتح الأمر لن يتبعوني.
 أبداً.
 أبداً.
 أبداً.
 لا تبعونا شيئاً.
 لأنك تبعونا.
 حسنًا.
 حسنًا.
 هناك لدينا أمام المنزلينة.
 حسنًا.
 حسنًا.
 حسنًا.
 حسنًا.
 حسنًا.
 حسنًا.
 أمام المنزلينة.
 بعد هذا ينبعكمكم في المنزلينة.
 حسنًا.
 فإنه لا يمكنك